In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import pickle
from sklearn.ensemble import RandomForestClassifier
df=pd.read_csv('insurance-2.csv')
df

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium
4,69,62.2,1.60,3.94000,True,Indore,retired,High
...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low


In [15]:
df['bmi']=df['weight']/df['height']**2
df
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]
     
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3
df['city_tier']=df['city'].apply(city_tier)  
df

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,city_tier
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High,49.227482,2
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low,30.189017,1
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low,21.118382,2
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium,45.535900,1
4,69,62.2,1.60,3.94000,True,Indore,retired,High,24.296875,2
...,...,...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low,21.420747,2
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low,47.984483,1
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low,18.765432,1
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low,30.521676,1


In [20]:
def lifestyle(row)-> str:
    if row['smoker'] and row['bmi'] > 30:
        return 'high'
    elif row['smoker'] and row['bmi']> 27:
        return 'medium'
    else:
        return 'low'
df['lifestyle']=df.apply(lifestyle,axis=1)  
df


,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,city_tier,lifestyle
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High,49.227482,2,low
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low,30.189017,1,low
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low,21.118382,2,low
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium,45.535900,1,high
4,69,62.2,1.60,3.94000,True,Indore,retired,High,24.296875,2,low
...,...,...,...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low,21.420747,2,low
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low,47.984483,1,low
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low,18.765432,1,low
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low,30.521676,1,low


In [26]:
def age_group(age)-> str:
    if age < 25:
        return 'young'
    elif age < 45:
        return 'adult'
    elif age < 60 :
        return 'middle-age'
    return 'old'
df['age_group']=df['age'].apply(age_group)  
df


,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,city_tier,lifestyle,age_group
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High,49.227482,2,low,old
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low,30.189017,1,low,adult
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low,21.118382,2,low,adult
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium,45.535900,1,high,young
4,69,62.2,1.60,3.94000,True,Indore,retired,High,24.296875,2,low,old
...,...,...,...,...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low,21.420747,2,low,adult
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low,47.984483,1,low,adult
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low,18.765432,1,low,middle-age
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low,30.521676,1,low,adult


In [34]:
X=df[['bmi','income_lpa','occupation','city_tier','lifestyle','age_group']]
X
y=df['insurance_premium_category']
y


,insurance_premium_category
0,High
1,Low
2,Low
3,Medium
4,High
...,...
95,Low
96,Low
97,Low
98,Low


In [42]:
categorical_data=['occupation','lifestyle','age_group','city_tier']
numerical_data=['bmi','income_lpa']               
preprocessing=ColumnTransformer([
    ('t1',OneHotEncoder(),categorical_data),
    ('t2','passthrough',numerical_data)
])
pipeline=Pipeline(steps=[
    ('preprocess',preprocessing),
    ('fc',RandomForestClassifier(random_state=42))
])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)
pipeline.fit(X_train,y_train)
pipeline.predict(X_test)
filename='model.pkl'
with open(filename,'wb') as f:
    pickle.dump(filename,f)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
